In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/phd_metabolomics/'

osp = MAC

## Load in limma results

In [8]:
toptable_ind = pd.read_csv(osp+'pymz/mzmine/221119/limma_toptable_independent.csv', sep=',')
feature_description = pd.read_csv(osp+'pymz/mzmine/221119/feature_description.csv', sep=',', index_col = 0)

In [11]:
#using the non-interaction model
data = feature_description.loc[toptable_ind.index]
#data['p-val'] = toptable_ind['P.Value']
data['adj p-val'] = toptable_ind['adj.P.Val']
data['t-score'] = toptable_ind['t']
data['custom_id'] = list(data.index)

In [ ]:
#save to csv for mummichog analysis
data.to_csv(osp+'pymz/mzmine/221119/data_for_mummichog.txt', header=True, index=None, sep='\t', mode='w')

In [21]:
data = pd.read_csv('/Users/anamaria/Desktop/mzmine/hat_topfeatures_filled.txt', sep='\t')

In [23]:
data.index = data['custom_id']

## Run mummichog

In [14]:
os.system('mummichog -f'+ osp + 'pymz/mzmine/221119/data_for_mummichog.txt'+' -m positive -u 3 -k MummichogResults -o mm -c 0.05')

NameError: name 'os' is not defined

## Annotate data

In [25]:
compounds = pd.read_csv('1588152677.696043.mm/tables/ListOfEmpiricalCompounds.tsv', sep='\t')
featureoutput = pd.read_csv('1588152677.696043.mm/tables/userInputData.txt', sep='\t')
pathway = pd.read_csv('1588152677.696043.mm/tables/mcg_pathwayanalysis_mm.tsv', sep='\t')

In [26]:
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    matching_ids[key] = value

In [27]:
compounds_dict = {}
for _,row in compounds.iterrows():
    ids = row['massfeature_rows']
    ions = row['str_row_ion']
    compoundlist = row['compound_names']
    
    ids_list = ids.split(';')
    ions_list = ions.split(';')
    for i in range(len(ids_list)):
        compounds_dict[matching_ids[ids_list[i]]] = []
        compounds_dict[matching_ids[ids_list[i]]].append(ions_list[i].split('_')[1])
        compounds_dict[matching_ids[ids_list[i]]].append(compoundlist)
        

In [28]:
for row_id,row in data.iterrows():
    if row_id in list(compounds_dict.keys()):
        
        data.at[row_id,'compound_names'] = compounds_dict[row_id][-1]
    else:
         data.at[row_id,'compound_names'] = 'NaN'

In [29]:
ecomp_dict = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    massfeature_rows = row['massfeature_rows'].split(';')
    for mass in massfeature_rows:
        if mass in ecomp_dict:
            ecomp_dict[mass].append(eid)
        else:
            ecomp_dict[mass] = []
            ecomp_dict[mass].append(eid)
            
cmpd_names = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    name = row['compound_names']
    cmpd_names[eid] = name
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    
    matching_ids[value] = key
for n in matching_ids:
    if matching_ids[n] in ecomp_dict:
        matching_ids[n] = ecomp_dict[matching_ids[n]]

id_names = {}
for i in matching_ids:
    if type(matching_ids[i]) is list:
        for f in matching_ids[i]:
            if f in cmpd_names:
                if i in id_names:
                    id_names[i].append(cmpd_names[f])
                else:
                    id_names[i] = []
                    id_names[i].append(cmpd_names[f])
matching_ids2 = {}
for n in matching_ids:
    
    if type(matching_ids[n]) is list:
        lista = matching_ids[n]
        for p in lista:
            if p in matching_ids2:
                matching_ids2[p].append(n)
            else:
                matching_ids2[p] = []
                matching_ids2[p].append(n)
pathway_dict = {}
for _,row in pathway.iterrows():
    
    pname = row['pathway']
    idlist =row['overlap_EmpiricalCompounds (id)']
    
    if type(idlist) != float:
        idlist = idlist.split(',')
        for p in idlist:
            
            if pname in pathway_dict:
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
            else:
                pathway_dict[pname] = []
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
pathway_dict2 = {}
for p in pathway_dict:
    
    for n in pathway_dict[p]:
        for i in n:
            if i in pathway_dict2:
                if p not in  pathway_dict2[i]:
                    pathway_dict2[i].append(p)
            else:
                pathway_dict2[i] = []
                pathway_dict2[i].append(p)
                
data['compound_names'] = str(np.zeros(len(data)))
data['compound_id'] = str(np.zeros(len(data)))
data['pathway'] = str(np.zeros(len(data)))
for row_id,row in data.iterrows():
    if row_id in id_names:
        data.at[row_id,'compound_names'] = id_names[row_id]    
    else:
        data.at[row_id,'compound_names'] = 'NaN'
    if row_id in matching_ids:
        data.at[row_id,'compound_id'] = matching_ids[row_id]
    else:
        data.at[row_id,'compound_id'] = 'NaN'
    if row_id in pathway_dict2:
        data.at[row_id,'pathway'] = pathway_dict2[row_id]
    else:
        data.at[row_id,'pathway'] = 'NaN'
    

In [30]:
data

,row m/z,row retention time,adj-p,t-val,custom_id,compound_names,compound_id,pathway
custom_id,,,,,,,,
7219,481.232518,7.884399,4.577734e-86,232.540494,7219,NaN,row1,NaN
3013,302.196068,10.477401,9.116643e-14,-11.712335,3013,NaN,row2,NaN
2092,202.180209,11.976537,1.553678e-13,-11.450319,2092,NaN,row3,NaN
2233,216.195819,10.855802,2.904047e-13,-11.197623,2233,NaN,row4,NaN
2106,203.183588,11.974607,4.188228e-13,-11.028958,2106,NaN,row5,NaN
2254,217.199121,10.851871,4.188228e-13,-10.947863,2254,NaN,row6,NaN
3086,317.215185,9.938294,4.188228e-13,-10.943577,3086,NaN,row7,NaN
3080,316.211756,9.939838,6.219595e-13,-10.799357,3080,NaN,row8,NaN
3136,330.227323,9.587620,9.209904e-13,-10.660827,3136,NaN,row9,NaN


## Matching with standards ToxID annotation

In [25]:
zik_standards_file_list = glob.glob(os.path.join(osp,'Metabolomics_datasets/Zika/', "*Std*.csv"))
zik_standards_file_list.sort()

zik_std1_csv_file = pd.read_csv(zik_standards_file_list[0], skiprows=9, index_col=False)
zik_std2_csv_file = pd.read_csv(zik_standards_file_list[2], skiprows=15, index_col=False)
zik_std3_csv_file = pd.read_csv(zik_standards_file_list[4], skiprows=15, index_col=False)


In [26]:
from metab_utils import create_std_dict

zik_standards_1 = create_std_dict(zik_std1_csv_file)
zik_standards_2 = create_std_dict(zik_std2_csv_file)
zik_standards_3 = create_std_dict(zik_std3_csv_file)

from metab_utils import mergeDict
zik_standards = mergeDict(mergeDict(zik_standards_1,zik_standards_2), zik_standards_3)


In [98]:
hat_standards

{'1-Naphthylacetic acid': ('187.07538', 5.45, '5.55', '+'),
 'Serotonin': ('177.10222', 15.28, '12.46', '+'),
 'Melatonin': ('233.12825', 5.84, '5.98', '+'),
 'Phenylhydrazine': ('109.07606', 6.01, '6.19', '+'),
 'Nicotinate': ('124.03927', 8.05, '7.73', '+'),
 'Pyridoxine': ('170.08109', 14.96, '13.34', '+'),
 'riboflavin': ('377.14520', 8.0, '8.19', '+'),
 'Glycerol': ('93.05466', 9.58, '9.66', '+'),
 'Adenine': ('136.06177', 14.02, '11.97', '+'),
 'Creatinine': ('114.06618', 15.63, '12.98', '+'),
 'L-Phenylalanine': ('166.08621', 11.67, '10.65', '+'),
 'L-Leucine': ('132.10185', 12.22, '10.90', '+'),
 '4-Aminobenzoate': ('138.05490', 6.16, '6.23', '+'),
 'Inosine': ('269.08777', 9.81, '9.93', '+'),
 '2-Phenylglycine': ('152.07057', 12.22, '11.35', '+'),
 'Selenomethionine': ('192.00868', 12.41, '11.33', '+'),
 'L-Methionine': ('150.05832', 12.58, '11.52', '+'),
 'Imidazole-4-acetate': ('127.05016', 14.79, '12.54', '+'),
 'L-Tryptophan': ('205.09715', 12.33, '11.03', '+'),
 'N-Acetyl

In [40]:
hat_standards_file_list = glob.glob(os.path.join(osp,'Metabolomics_datasets/HAT_serum/', "*Std*.csv"))
hat_standards_file_list.sort()

hat_std1_csv_file = pd.read_csv(hat_standards_file_list[0], skiprows=8, index_col=False)
hat_std2_csv_file = pd.read_csv(hat_standards_file_list[2], skiprows=8, index_col=False)
hat_std3_csv_file = pd.read_csv(hat_standards_file_list[4], skiprows=8, index_col=False)


In [43]:
from metab_utils import create_std_dict

hat_standards_1 = create_std_dict(hat_std1_csv_file)
hat_standards_2 = create_std_dict(hat_std2_csv_file)
hat_standards_3 = create_std_dict(hat_std3_csv_file)

from metab_utils import mergeDict
hat_standards = mergeDict(mergeDict(hat_standards_1,hat_standards_2), hat_standards_3)

In [81]:
standards['L-Cystine'] = ('241.03096', 16.47, '16.01', '+')

In [82]:
#filtered_peaks['standards_match'] = ['']*len(filtered)
standards = hat_standards
for i,row in data.iterrows():
    mz = float(row['row m/z'])
    rt = float(row['row retention time'])
    
    for metabolite in standards:
        if standards[metabolite][0] != 'N':
            mz_std = float(standards[metabolite][0])
            rt_std = float(standards[metabolite][2])
        
       
            if mz >= mz_std - 0.001 and mz <= mz_std + 0.001:
                if rt >= rt_std - 0.2 and rt <= rt_std + 0.2:
                    data.at[i,'standards_match'] = metabolite
                else:
                    data.at[i,'standards_match'] = 'NaN'


In [29]:
data.to_csv(osp+'pymz/mzmine/221119/annotated_data.csv', sep = ',')

In [96]:
data.to_csv('/Users/anamaria/Desktop/mzmine/221119/hat_annotated_data.csv', sep = ',')

## Check accuracy of mummichog annotation

In [94]:
notannotated = 0
annotated = 0
notannotateds = 0
annotateds = 0
match = 0
matchs =0
for i,row in data.iterrows():
    cmp = str(row['compound_names']).lower()
    std = str(row['standards_match']).lower()
    
    if cmp in ['nan']:
        notannotated +=1
        if not std in ['nan']:
            matchs +=1
           
    else:
        annotated +=1
        if not std in ['nan']:
            if std in cmp:
                match +=1
            else:
                print(row)
    
    if std in ['nan']:
        notannotateds +=1
    else:
        annotateds +=1
        
percentage_exact_matches = match*100/annotateds
percentage_not_annotated = matchs*100/annotateds
percentage_not_matched = (annotateds-match-matchs)*100/annotateds

print("Total number of annotated compounds using mummichog: {}".format(annotated))
print("Total number of annotated compounds using ToxID standards: {}".format(annotateds))
print("Total number of annotation matches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(match, percentage_exact_matches))
print("Total number of annotation mismatches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(annotateds-match-matchs, percentage_not_matched))
print("Total number of metabolites annotated with ToxID, but not with mummichog: {0}, i.e. {1:.2f}%".format(matchs, percentage_not_annotated))

row m/z                                                         133.097
row retention time                                              18.1115
adj-p                                                          0.412094
t-val                                                          -1.40793
custom_id                                                          1065
compound_names        [Ornithine$D-Ornithine$Ornithine; 2,5-Diaminov...
compound_id                                                [E433, E920]
pathway                                                             NaN
standards_match                                             L-Ornithine
Name: 1065, dtype: object
row m/z                        136.043
row retention time             12.1061
adj-p                         0.579975
t-val                            1.039
custom_id                         5759
compound_names        [L-Homocysteine]
compound_id                     [E331]
pathway                            NaN
standards_matc